In [3]:
!pip install selenium
from selenium import webdriver
#!pip install BeautifulSoup
from bs4 import BeautifulSoup
import os
import io
import time
import re
import random
import pandas as pd
import calendar
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options




In [10]:
#### INPUTS #################################################
#next_sf = "a.ancBtn sml green icon iconArrowRight"
#next_ny = "a.pagingNext"
NY = "https://stevemorse.org/ellis/passengers.php?mode=ny"
SF = "https://stevemorse.org/ellis/passengers.php?mode=sanfran"

immigrant_page = NY # Change this depending on which ones you need

# Credentials --  change if free trial runs out
user = "yy2657@columbia.edu"
password = "StarHeart1998!"

# Input max number of pages. -- usually max results divided by 50
#max_value = 9858

# Year to input:
year = '1945' # Change to 1887 or 1945 depending on need
months =  ['12']#
#days = [str(i) for i in range(11, 28)] #1-15, 16-eom
port_entry = "NY" # Change this if needed - it's used for naming the files

In [11]:
for month in months:
    num_days = calendar.monthrange(int(year), int(month))[1]
    days = [str(i) for i in range(1, num_days+1)]
    for day in range(1, num_days+1):
    #### Log In #####
        # Open log in page
        tries = 0
        driver = webdriver.Chrome(executable_path="./chromedriver_mac64/chromedriver")
        page = driver.get('https://www.ancestry.com/account/signin?returnUrl=')

        # Log in
        driver.implicitly_wait(10)
        wait = WebDriverWait(driver, 10) 
        element = wait.until(EC.presence_of_element_located((By.ID, 'username')))
        element.click()
        element.send_keys(user)
        element = wait.until(EC.presence_of_element_located((By.ID, 'password')))
        element.click()
        element.send_keys(password)

        element = wait.until(EC.presence_of_element_located((By.ID, 'signInBtn')))
        element.click()
        time.sleep(5)
        driver.get(immigrant_page) # INPUT 
            # Select the year
        year_select = Select(driver.find_element(By.NAME, 'yr'))
        year_select.select_by_value(year)

        # Select the month
        mo_select = Select(driver.find_element(By.NAME, 'mo'))
        mo_select.select_by_value(month)

        # Select the day
        day_select = Select(driver.find_element(By.NAME, 'da'))
        day_select.select_by_value(str(day))


        ## Select the yrd
        # Find the search button
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="button"][value="search"]')))
        time.sleep(3)
        element.click()

        # Switch to the opened tab
        p = driver.current_window_handle
        parent = driver.window_handles[0]
        chld = driver.window_handles[1]
        driver.switch_to.window(chld)

        time.sleep(10)
        # Begin Iterating and Saving Dataframes:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        error_message = soup.find('div', {'class': 'searchError'})

        if error_message is None:
            while True:
                
                try:
                    counter = 0
                    data = []
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                
                    max_element = soup.find('span', {'class': 'pagingMax'})
                    max_value = int(max_element.text.replace(',', ''))
                    max_value = min(max_value, 100)
                    print(max_value)
                    # break out of the loop if the button is clicked successfully
                    break
                except AttributeError as e:

                    time.sleep(5)
                    print(f"Error: {e}. Refreshing...")
                    driver.refresh()
            
            #max_value
            for i in range(1, max_value):
                counter = counter + 1
                wait = WebDriverWait(driver, 10)
                wait.until(EC.presence_of_element_located((By.ID, 'collection-results-table')))
                html = driver.page_source

                # Parse HTML with Beautiful Soup
                soup = BeautifulSoup(html, 'html.parser')

                # Find the table element by ID
                table = soup.find('table', {'id': 'collection-results-table'})
                # Extract the table rows
                rows = table.find_all('tr')


                for row in rows[1:]:
                    cells = row.find_all('td')
                    data.append({
                        'Name': cells[1].text,
                        'Birth Year': cells[2].text.strip(),
                        'Arrival Date': cells[3].text.strip(),
                        'Departure Port': cells[4].text.strip(),
                        'Ethnicity/Nationality': cells[5].text.strip(),
                        'Ship Name': cells[6].text.strip()
                    })

                df = pd.DataFrame(data)
                
                print(year,"-",month,"-",day,"-",counter)
                
                time.sleep(random.randint(1, 5))
                if counter % 10 == 0:
                    df.to_csv(f'csvs/archive/data_{port_entry}_{year}_{month}_{day}_{counter}.csv', index=False)

                if i != max_value:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'a.pagingNext')
                    next_button.click()
                # Add random wait time between 1-5 seconds

            df.to_csv(f'csvs/data_{port_entry}_{year}_{month}_{day}_{counter}_final.csv', index=False)
            driver.quit()
        else: 
            driver.quit()

/var/folders/53/zn46v9s96111cklvxsj1blcw0000gn/T/ipykernel_10656/2448013875.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="./chromedriver_mac64/chromedriver")


45
1945 - 12 - 1 - 1
1945 - 12 - 1 - 2
1945 - 12 - 1 - 3
1945 - 12 - 1 - 4
1945 - 12 - 1 - 5
1945 - 12 - 1 - 6
1945 - 12 - 1 - 7
1945 - 12 - 1 - 8
1945 - 12 - 1 - 9
1945 - 12 - 1 - 10
1945 - 12 - 1 - 11
1945 - 12 - 1 - 12
1945 - 12 - 1 - 13
1945 - 12 - 1 - 14
1945 - 12 - 1 - 15
1945 - 12 - 1 - 16
1945 - 12 - 1 - 17
1945 - 12 - 1 - 18
1945 - 12 - 1 - 19
1945 - 12 - 1 - 20
1945 - 12 - 1 - 21
1945 - 12 - 1 - 22
1945 - 12 - 1 - 23
1945 - 12 - 1 - 24
1945 - 12 - 1 - 25
1945 - 12 - 1 - 26
1945 - 12 - 1 - 27
1945 - 12 - 1 - 28
1945 - 12 - 1 - 29
1945 - 12 - 1 - 30
1945 - 12 - 1 - 31
1945 - 12 - 1 - 32
1945 - 12 - 1 - 33
1945 - 12 - 1 - 34
1945 - 12 - 1 - 35
1945 - 12 - 1 - 36
1945 - 12 - 1 - 37
1945 - 12 - 1 - 38
1945 - 12 - 1 - 39
1945 - 12 - 1 - 40
1945 - 12 - 1 - 41
1945 - 12 - 1 - 42
1945 - 12 - 1 - 43
1945 - 12 - 1 - 44
Error: 'NoneType' object has no attribute 'text'. Refreshing...
100
1945 - 12 - 2 - 1
1945 - 12 - 2 - 2
1945 - 12 - 2 - 3
1945 - 12 - 2 - 4
1945 - 12 - 2 - 5
1945 - 12 - 

TimeoutException: Message: timeout: Timed out receiving message from renderer: -335.740
  (Session info: chrome=112.0.5615.49)
Stacktrace:
0   chromedriver                        0x00000001027d0428 chromedriver + 4899880
1   chromedriver                        0x000000010274da23 chromedriver + 4364835
2   chromedriver                        0x0000000102397bf6 chromedriver + 474102
3   chromedriver                        0x00000001023804d9 chromedriver + 378073
4   chromedriver                        0x000000010238008b chromedriver + 376971
5   chromedriver                        0x000000010237e8ba chromedriver + 370874
6   chromedriver                        0x000000010237edec chromedriver + 372204
7   chromedriver                        0x000000010238db2c chromedriver + 432940
8   chromedriver                        0x000000010238ee32 chromedriver + 437810
9   chromedriver                        0x00000001023a0794 chromedriver + 509844
10  chromedriver                        0x0000000

In [20]:
year = '1887' # Change to 1887 or 1945 depending on need
month =  '12'#
for day in range(8,16):
    #### Log In #####
        # Open log in page
        tries = 0
        driver = webdriver.Chrome(executable_path="./chromedriver_mac64/chromedriver")
        page = driver.get('https://www.ancestry.com/account/signin?returnUrl=')

        # Log in
        driver.implicitly_wait(10)
        wait = WebDriverWait(driver, 10) 
        element = wait.until(EC.presence_of_element_located((By.ID, 'username')))
        element.click()
        element.send_keys(user)
        element = wait.until(EC.presence_of_element_located((By.ID, 'password')))
        element.click()
        element.send_keys(password)

        element = wait.until(EC.presence_of_element_located((By.ID, 'signInBtn')))
        element.click()
        time.sleep(5)
        driver.get(immigrant_page) # INPUT 
            # Select the year
        year_select = Select(driver.find_element(By.NAME, 'yr'))
        year_select.select_by_value(year)

        # Select the month
        mo_select = Select(driver.find_element(By.NAME, 'mo'))
        mo_select.select_by_value(month)

        # Select the day
        day_select = Select(driver.find_element(By.NAME, 'da'))
        day_select.select_by_value(str(day))


        ## Select the yrd
        # Find the search button
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="button"][value="search"]')))
        time.sleep(3)
        element.click()

        # Switch to the opened tab
        p = driver.current_window_handle
        parent = driver.window_handles[0]
        chld = driver.window_handles[1]
        driver.switch_to.window(chld)

        time.sleep(10)
        # Begin Iterating and Saving Dataframes:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        error_message = soup.find('div', {'class': 'searchError'})

        if error_message is None:
            while True:
                
                try:
                    counter = 0
                    data = []
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                
                    max_element = soup.find('span', {'class': 'pagingMax'})
                    max_value = int(max_element.text.replace(',', ''))
                    max_value = min(max_value, 100)
                    print(max_value)
                    # break out of the loop if the button is clicked successfully
                    break
                except AttributeError as e:

                    time.sleep(5)
                    print(f"Error: {e}. Refreshing...")
                    driver.refresh()
            
            #max_value
            for i in range(1, max_value):
                counter = counter + 1
                wait = WebDriverWait(driver, 10)
                wait.until(EC.presence_of_element_located((By.ID, 'collection-results-table')))
                html = driver.page_source

                # Parse HTML with Beautiful Soup
                soup = BeautifulSoup(html, 'html.parser')

                # Find the table element by ID
                table = soup.find('table', {'id': 'collection-results-table'})
                # Extract the table rows
                rows = table.find_all('tr')


                for row in rows[1:]:
                    cells = row.find_all('td')
                    data.append({
                        'Name': cells[1].text,
                        'Birth Year': cells[2].text.strip(),
                        'Arrival Date': cells[3].text.strip(),
                        'Departure Port': cells[4].text.strip(),
                        'Ethnicity/Nationality': cells[5].text.strip(),
                        'Ship Name': cells[6].text.strip()
                    })

                df = pd.DataFrame(data)
                
                print(year,"-",month,"-",day,"-",counter)
                
                time.sleep(random.randint(1, 5))
                if counter % 10 == 0:
                    df.to_csv(f'csvs/archive/data_{port_entry}_{year}_{month}_{day}_{counter}.csv', index=False)

                if i != max_value:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'a.pagingNext')
                    next_button.click()
                # Add random wait time between 1-5 seconds

            df.to_csv(f'csvs/data_{port_entry}_{year}_{month}_{day}_{counter}_final.csv', index=False)
            driver.quit()
        else: 
            driver.quit()

/var/folders/53/zn46v9s96111cklvxsj1blcw0000gn/T/ipykernel_10656/1635570904.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="./chromedriver_mac64/chromedriver")


Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribute 'text'. Refreshing...
Error: 'NoneType' object has no attribut

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.49)
Stacktrace:
0   chromedriver                        0x00000001033e3428 chromedriver + 4899880
1   chromedriver                        0x0000000103360a23 chromedriver + 4364835
2   chromedriver                        0x0000000102faabf6 chromedriver + 474102
3   chromedriver                        0x0000000102f80cdc chromedriver + 302300
4   chromedriver                        0x000000010301985f chromedriver + 927839
5   chromedriver                        0x000000010302f226 chromedriver + 1016358
6   chromedriver                        0x0000000103014333 chromedriver + 906035
7   chromedriver                        0x0000000102fde55f chromedriver + 685407
8   chromedriver                        0x0000000102fdfa7e chromedriver + 690814
9   chromedriver                        0x00000001033b079e chromedriver + 4691870
10  chromedriver   